In [1]:
from pathlib import Path
import logging
from typing import Optional, Dict, Any
import json

from geoworkflow.schemas.config_models import AOIConfig
from geoworkflow.processors.aoi.processor import AOIProcessor
from geoworkflow.utils.earth_engine_utils import EarthEngineAuth, OpenBuildingsAPI
from geoworkflow.core.exceptions import EarthEngineError, EarthEngineAuthenticationError

In [2]:
"""SEMI-OPTIONAL
This area demonstrates how to specify an area of interest (AOI). Specifically, it generates an AOI
of the Accra, Ghana, shape defined in AFRICAPOLIS2020.geojson
For demonstration purposes, the 'aoi' file name might be changed in the following blocks.
"""
aoi = "../data/aoi/accra_aoi.geojson"
# Create AOI configuration for Accra
config = AOIConfig(
    input_file=Path("../data/01_extracted/AFRICAPOLIS2020.geojson"),
    country_name_column="agglosName",  # Use agglosName instead of country column
    countries=["Accra"],  # Filter for Accra agglomeration
    buffer_km=0,  # No buffer 
    dissolve_boundaries=False,  # Keep original feature structure
    output_file=Path(aoi)
)

# Create and run the processor
processor = AOIProcessor(config)
result = processor.process()

# Check results
if result.success:
    print(f"✅ {result.message}")
    print(f"Processing time: {result.elapsed_time:.2f}s")
    print(f"Output saved to: {config.output_file}")
    
    # Display metadata
    if result.metadata:
        print("\nMetadata:")
        for key, value in result.metadata.items():
            print(f"  {key}: {value}")
else:
    print(f"❌ Processing failed: {result.message}")

Output()

✅ Successfully created AOI with 1 features
Processing time: 5.29s
Output saved to: ../data/aoi/accra_aoi.geojson

Metadata:
  output_file: ../data/aoi/accra_aoi.geojson
  feature_count: 1
  countries_processed: 1
  buffer_applied_km: 0.0
  boundaries_dissolved: False
  output_crs: EPSG:4326


In [3]:
"""OPTIONAL:
Create a status logging tracker to monitor progress.
Users can create code without including this "logging" functionality.
The logging statements work like print staments, but can also save to file. 
They're helpful for debugging and tracking execution in larger workflows.

If you remove this block, make sure to also remove related logging calls in the code.
"""
# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [4]:
"""
This step fetches the authorization details to connect to Google Earth Engine.
The auth script is not shared on Github.
You will have to either create your own and place it in the config folder (make sure to change the name accordingly),
or contact the author to get access to the file. 
"""

auth_path = Path("../config/auth/africapolis-005f188d49d6.json")
with open(auth_path, 'r') as f:
        auth_data:dict = json.load(f)

In [ ]:
"""
This block initiates the Earth Engine authentication.  
"""

logger.info("🔐 Authenticating with Earth Engine...")

"""The try-except block captures and logs errors during authentication, querying, and exporting.
try-except blocks are OPTIONAL but you'll need to fix indentation if you remove them."""
try: 
    
    # Start authetication.
    authenticated_project_id = EarthEngineAuth.authenticate(
        service_account_key=auth_path,
        service_account_email=auth_data["client_email"],
        project_id=auth_data["project_id"]
    )
    logger.info(f"✅ Authenticated with project: {authenticated_project_id}")

    """OPTIONAL: see note above try above"""
except EarthEngineAuthenticationError as e:
    logger.error(f"❌ Authentication failed: {e}")
    print({
        "status": "error", 
        "error_type": "AuthenticationError", 
        "message": str(e),
        "help": "Check your service account key and email configuration"
    })
except Exception as e:
    logger.error(f"❌ Unexpected error: {e}")
    print({
        "status": "error", 
        "error_type": "GeneralError", 
        "message": str(e),
        "help": "Check file paths and ensure all dependencies are installed"
    })

2025-09-27 09:32:29,926 - INFO - 🚀 Starting Earth Engine building download...
2025-09-27 09:32:29,927 - INFO - 📍 Input area: ../data/aoi/accra_aoi.geojson
2025-09-27 09:32:29,928 - INFO - 💾 Output file: ../data/02_clipped/accra/accra_buildings.geojson
2025-09-27 09:32:29,928 - INFO - 🎯 Confidence threshold: 0.75
2025-09-27 09:32:29,929 - INFO - 🔐 Authenticating with Earth Engine...
2025-09-27 09:32:29,929 - INFO - Using service account: africapolis-buildings-extracto@africapolis.iam.gserviceaccount.com
2025-09-27 09:32:31,060 - INFO - Earth Engine authenticated with service account, project: africapolis
2025-09-27 09:32:31,060 - INFO - ✅ Authenticated with project: africapolis


In [7]:
"""
This block initializes the buildings API.
"""


"""The try-except block captures and logs errors during authentication, querying, and exporting.
try-except blocks are OPTIONAL but you'll need to fix indentation if you remove them."""
try:
    # Start: Initialize Open Buildings API
    logger.info("🏗️ Initializing Open Buildings API...")
    buildings_api = OpenBuildingsAPI(
        project_id=authenticated_project_id,
        dataset_version="v3"
    )
    
except Exception as e:
    logger.error(f"❌ Unexpected error: {e}")
    result_summary = {
        "status": "error", 
        "error_type": "GeneralError", 
        "message": str(e),
        "help": "Check file paths and ensure all dependencies are installed"
    }
    print(result_summary)

2025-09-27 09:32:31,073 - INFO - 🏗️ Initializing Open Buildings API...
2025-09-27 09:32:31,074 - INFO - Initialized Open Buildings API for dataset: GOOGLE/Research/open-buildings/v3/polygons


In [ ]:
confidence_threshold = 0.75  # OPTIONAL - confidence threshold for building detection (0.75 is default and recommended)
max_features = None # OPTIONAL - limit number of buildings downloaded (None is the default, meaning no limit. No limits, bro)

format_type = "geojson"

"""aoi, building_output = ../data/aoi/accra_sample_aoi.geojson', "../data/02_clipped/accra/accra_sample_buildings.geojson"
 is a smaller sample area for testing and demonstration purposes. Comment the block if you want to run the full Accra area.
 """
building_output = "../data/02_clipped/accra/accra_buildings.geojson"
aoi, building_output = "../data/aoi/accra_sample_aoi.geojson", "../data/02_clipped/accra/accra_sample_buildings.geojson"


logger.info("🚀 Starting Earth Engine building download...")
logger.info(f"📍 Input area: {aoi}")
logger.info(f"💾 Output file: {building_output}")
logger.info(f"🎯 Confidence threshold: {confidence_threshold}")

In [ ]:

"""
This block loads the buildings within the area of interest (AOI). 
The majority of the work is done in the export function, which can take a little over an hour for a city the size of Accra.
"""


"""The try-except block captures and logs errors during authentication, querying, and exporting.
try-except blocks are OPTIONAL but you'll need to fix indentation if you remove them."""
try:
    logger.info("📋 Loading area of interest...")
    aoi_geometry = buildings_api.load_aoi_geometry(Path(aoi))
    

    # Start: Filter buildings by confidence and area
    logger.info(f"🔍 Querying buildings with confidence >= {confidence_threshold}...")
    buildings = (buildings_api.collection
                .filterBounds(aoi_geometry)
                .filter(f'confidence >= {confidence_threshold}'))
    
    buildings = buildings.filter('area_in_meters >= 5')      # Min 10 sqm
    buildings = buildings.filter('area_in_meters <= 100000')  # Max 100k sqm
    
    
    # Step 5: Export with current API including grid processing
    logger.info(f"💾 Exporting to {format_type} format...")
    
    buildings_api.export_to_format(
        collection=buildings,
        output_path=Path(building_output),
        format_type=format_type,
        include_properties=['confidence', 'area_in_meters', 'plus_code'],
        max_features=max_features,
    )
    
    # Step 6: Verify output and create summary
    output_file = Path(building_output)
    if output_file.exists():
        file_size_mb = building_output.stat().st_size / (1024 * 1024)
        logger.info(f"✅ Export completed! File size: {file_size_mb:.2f} MB")
    else:
        logger.warning("⚠️ Output file not found - export may have failed")
    
    # Create result summary
    result_summary = {
        "status": "success",
        "input_area": aoi,
        "output_file": building_output,
        "output_format": format_type,
        "confidence_threshold": confidence_threshold,
        "project_id": authenticated_project_id,
        "file_size_mb": round(file_size_mb, 2) if output_file.exists() else None
    }
    
    logger.info("🎉 Building download completed successfully!")
    

except EarthEngineError as e:
    logger.error(f"❌ Earth Engine error: {e}")
    result_summary = {
        "status": "error", 
        "error_type": "EarthEngineError", 
        "message": str(e),
        "help": "Try reducing the area size or increasing confidence threshold"
    }
except Exception as e:
    logger.error(f"❌ Unexpected error: {e}")
    result_summary = {
        "status": "error", 
        "error_type": "GeneralError", 
        "message": str(e),
        "help": "Check file paths and ensure all dependencies are installed"
    }

print(result_summary)

2025-09-27 09:37:30,910 - INFO - 📋 Loading area of interest...
2025-09-27 09:37:30,913 - INFO - Dissolving 2 AOI features into single geometry
2025-09-27 09:37:31,068 - INFO - AOI geometry loaded successfully, area: 31,410,187 square meters
2025-09-27 09:37:31,069 - INFO - 🔍 Querying buildings with confidence >= 0.75...
2025-09-27 09:37:31,069 - INFO - 💾 Exporting to geojson format...
2025-09-27 09:37:31,070 - INFO - Counting features in collection...
2025-09-27 09:37:37,010 - INFO - Collection contains 28,117 features
2025-09-27 09:37:37,012 - INFO - Large dataset detected (28,117 features). Using grid-based export...
2025-09-27 09:37:37,012 - INFO - Starting grid-based export processing...
2025-09-27 09:37:44,614 - INFO - Created 15 grid cells for processing
2025-09-27 09:38:03,022 - INFO - Progress: 10/15 cells (66.7%)
2025-09-27 09:38:09,367 - INFO - Collected 28,385 features from 15 grid cells
2025-09-27 09:38:09,367 - INFO - Saving 28,385 features to geojson format...
2025-09-27 

{'status': 'error', 'error_type': 'GeneralError', 'message': "'str' object has no attribute 'stat'", 'help': 'Check file paths and ensure all dependencies are installed'}
